In [1]:
import json
import os
import sys
sys.path.insert(0, '../../..')
import networkx as nx

import pandas as pd

from scipy.misc import bytescale

from autocnet.examples import get_path
from autocnet.fileio.io_gdal import GeoDataset
from autocnet.fileio.io_controlnetwork import to_isis
from autocnet.graph.network import CandidateGraph
from autocnet.matcher import feature_extractor as fe
from autocnet.matcher.matcher import FlannMatcher
from autocnet.control.control import C

import cv2
print(cv2.__version__)
print(cv2.__file__)
from IPython.core.display import display

%pylab inline

3.0.0-dev
/Users/jlaura/anaconda3/envs/autocnet/lib/python3.5/site-packages/cv2.cpython-35m-darwin.so
Populating the interactive namespace from numpy and matplotlib


In [94]:
fl = FlannMatcher()
adjacency = get_path('three_image_adjacency.json')
basepath = os.path.dirname(adjacency)
cg = CandidateGraph.from_adjacency(adjacency)
for node, attributes in cg.nodes_iter(data=True):
    dataset = GeoDataset(os.path.join(basepath, attributes['image_name']))
    attributes['handle'] = dataset
    img = bytescale(dataset.read_array())
    attributes['image'] = img
    attributes['keypoints'], attributes['descriptors'] = fe.extract_features(attributes['image'],
                                                                             {'nfeatures':10})
    fl.add(attributes['descriptors'], key=node)
fl.train()

In [95]:
for node, attributes in cg.nodes_iter(data=True):
    descriptors = attributes['descriptors']
    matches = fl.query(descriptors, node,  k=4)
    display(matches)
    cg.add_matches(matches)

,source_image,source_idx,destination_image,destination_idx,distance
0,0,0,2,9,238.813736
1,0,0,2,1,242.606674
2,0,1,1,6,219.868149
3,0,2,1,7,65.551506
4,0,2,1,0,200.753586
5,0,2,1,1,207.761887
6,0,3,1,9,66.678329
7,0,3,1,0,214.492432
8,0,4,1,5,102.722931
9,0,4,1,6,155.083847


,source_image,source_idx,destination_image,destination_idx,distance
0,1,1,2,5,152.220230
1,1,2,2,1,68.139565
2,1,3,2,3,111.874931
3,0,6,1,3,196.318619
4,0,7,1,4,90.116592
5,0,4,1,4,245.316116
6,0,4,1,5,102.722931
7,0,6,1,6,64.521317
8,1,6,2,9,88.932556
9,0,2,1,7,65.551506


,source_image,source_idx,destination_image,destination_idx,distance
0,0,2,2,0,259.615112
1,1,7,2,0,268.534912
2,1,3,2,0,309.636871
3,1,2,2,1,68.139565
4,1,0,2,1,159.317291
5,1,6,2,1,164.386734
6,1,0,2,2,188.462204
7,1,8,2,2,189.261719
8,1,1,2,2,209.864243
9,1,3,2,3,111.874931


In [96]:
cnet = None
pt_type = 2
ms_type = 2

merged_cnet = None

for source, destination, attributes in cg.edges_iter(data=True):
    matches = attributes['matches']
    kp1 = cg.node[source]['keypoints']
    kp2 = cg.node[destination]['keypoints']
    
    # Symmetry check right here
    duplicates = matches.duplicated(keep='first')
    clean = matches[duplicates]
    
    pt_idx = 0
    measures = []
    values = []
    for idx, row in clean.iterrows():
        m1 = (source, int(row['source_idx']))
        m2 = (destination, int(row['destination_idx']))
        measures.append(m1)
        measures.append(m2)
        
        #TODO: We can build more complex values here
        values.append([kp1[m1[1]].pt[0], kp1[m1[1]].pt[1], m1, pt_idx, source])
        values.append([kp2[m2[1]].pt[0], kp2[m2[1]].pt[1], m2, pt_idx, destination])
        pt_idx += 1
        
    columns = ['x', 'y', 'idx', 'pid', 'nid']
    cnet = C(values, columns=columns)
    if merged_cnet is None:
        merged_cnet = cnet.copy(deep=True)
    else:
        pid_offset = merged_cnet['pid'].max() + 1  # Get the current max point index
        cnet[['pid']] += pid_offset
        
        #Inner merge on the dataframe identifies common points
        common = pd.merge(merged_cnet, cnet, how='inner', on='idx', left_index=True, suffixes=['_r', 
                                                                                              '_l'])

        #Iterate over the points to be merged and merge them in.
        for i, r in common.iterrows():
            new_pid = r['pid_r']
            update_pid = r['pid_l']
            cnet.loc[cnet['pid'] == update_pid, ['pid']] = new_pid  # Update the point ids
                
        #Perform the concat
        merged_cnet = pd.concat([merged_cnet, cnet])     
        merged_cnet.drop_duplicates(['idx', 'pid'], keep='first', inplace=True)
        
display(merged_cnet.sort_values(by='pid'))


,x,y,idx,pid,nid
0,767.366211,947.357056,"(0, 7)",0,0
1,977.130676,940.599792,"(1, 4)",0,1
2,756.705078,270.404785,"(0, 4)",1,0
3,967.310730,273.262665,"(1, 5)",1,1
4,245.627838,973.774841,"(0, 6)",2,0
5,453.814636,974.998169,"(1, 6)",2,1
3,662.654175,975.702576,"(2, 9)",2,2
6,697.417786,919.242798,"(0, 2)",3,0
7,909.761230,914.576477,"(1, 7)",3,1
8,697.417786,919.242798,"(0, 3)",4,0


In [ ]:
q

In [ ]:
nx.draw(G)
show()

In [ ]:
#Add single edges to the graph
basename = 'AS15-M-0{}_SML.png'
for k,v in adjacency.iteritems():
    for i in v:
        G.add_edge(k,i)

In [ ]:
nx.draw(G, label=G.nodes())
show()

In [ ]:
G.adjacency_to_json('/Users/jlaura/github/autocnet/autocnet/examples/Apollo15/adjacency.json')

In [ ]:
G2 = network.CandidateGraph.from_adjacency('/Users/jlaura/github/autocnet/autocnet/examples/Apollo15/adjacency.json')

In [ ]:
nx.draw(G2)

In [ ]:
for n in G.nodes():
    print n, G.neighbors(n)

In [ ]:
G.edges()

In [ ]:
#Perform a check to see if the subgraph is connected
print nx.is_strongly_connected(G)

In [ ]:
#Get a list fo the nodes in each subgraph, no connectivity
print len(list(nx.strongly_connected_components(G)))

In [ ]:
#Return the graph objecets for the sub-graphs
for sg in nx.strongly_connected_component_subgraphs(G, copy=False):
    print sg.nodes()